## CellProfiler-OMERO demo
We have setup an example notebook to guide you through the steps to analise images stored in OMERO using CellProfiler

Let's start by importing some libraries we are going to need

In [1]:
import numpy as np
from ipywidgets import IntProgress
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

import ezomero

import cellprofiler_core.preferences as cp_preferences
import cellprofiler_core.pipeline as cp_pipeline
import cellprofiler_core.measurement as cp_measurement
from cellprofiler_core.modules.injectimage import InjectImage

import pandas as pd
import tempfile


Bad key text.latex.preview in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /home/julio/.conda/envs/cellprofiler4/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95    

In [2]:
# Make CellProfiler run without a GUI
cp_preferences.set_headless()

# Tell CellProfiler to get input from and save output in a temp directory
output_dir = tempfile.TemporaryDirectory()
input_dir = tempfile.TemporaryDirectory()
cp_preferences.set_default_output_directory(output_dir.name)
cp_preferences.set_default_image_directory(input_dir.name)

In [14]:
import omero_rois
from omero.gateway import ColorHolder
from omero.model import MaskI, RoiI
from omero.rtypes import rstring, rdouble, rint


def create_roi(conn, img, shapes, name):
    updateService = conn.getUpdateService()

    # create an ROI, link it to Image
    roi = RoiI()
    # use the omero.model.ImageI that underlies the 'image' wrapper
    roi.setImage(img._obj)
    roi.setName(rstring(name))
    for shape in shapes:
        # shape.setTextValue(rstring(name))
        roi.addShape(shape)
    # Save the ROI (saves any linked shapes too)
    return updateService.saveAndReturnObject(roi).id.val


def masks_from_labels_image_3d(
        labels_3d, rgba=None, c=None, t=None, text=None,
        raise_on_no_mask=True):  # sourcery skip: low-code-quality
    """
    Create a mask shape from a binary image (background=0)

    :param numpy.array labels_3d: labels 3D array
    :param rgba int-4-tuple: Optional (red, green, blue, alpha) colour
    :param c: Optional C-index for the mask
    :param t: Optional T-index for the mask
    :param text: Optional text for the mask
    :param raise_on_no_mask: If True (default) throw an exception if no mask
           found, otherwise return an empty Mask
    :return: An OMERO mask
    :raises NoMaskFound: If no labels were found
    :raises InvalidBinaryImage: If the maximum labels is greater than 1
    """
    shapes = {}
    for i in range(1, labels_3d.max() + 1):
        if not np.any(labels_3d == i):
            continue

        masks = []
        bin_img = labels_3d == i
        # Find bounding box to minimise size of mask
        xmask = bin_img.sum(0).sum(0).nonzero()[0]
        ymask = bin_img.sum(0).sum(1).nonzero()[0]
        if any(xmask) and any(ymask):
            x0 = min(xmask)
            w = max(xmask) - x0 + 1
            y0 = min(ymask)
            h = max(ymask) - y0 + 1
            submask = bin_img[:, y0:(y0 + h), x0:(x0 + w)]
        else:
            if raise_on_no_mask:
                raise omero_rois.NoMaskFound()
            x0 = 0
            w = 0
            y0 = 0
            h = 0
            submask = []

        for z, plane in enumerate(submask):
            if np.any(plane):
                mask = MaskI()
                mask.setBytes(np.packbits(np.asarray(plane, dtype=int)))
                mask.setWidth(rdouble(w))
                mask.setHeight(rdouble(h))
                mask.setX(rdouble(x0))
                mask.setY(rdouble(y0))
                mask.setTheZ(rint(z))

                if rgba is not None:
                    ch = ColorHolder.fromRGBA(*rgba)
                    mask.setFillColor(rint(ch.getInt()))
                if c is not None:
                    mask.setTheC(rint(c))
                if t is not None:
                    mask.setTheT(rint(t))
                if text is not None:
                    mask.setTextValue(rstring(text))

                masks.append(mask)

        shapes[i] = masks

    return shapes


Let's connect to OMERO. When we connect we get a connection object that we will have to use in every interaction with OMERO.

In [3]:
# Creating a connection object
host = "omero.mri.cnrs.fr"
port = 4064
conn = ezomero.connect(host=host, port=port)

# Connecting
conn.connect()
# The connection will timeout after a period of inactivity. To avoid that we can tell our new connection to say "Hi, I'm still here"
conn.c.enableKeepAlive(60)
# Let's verify that we are connected
conn.isConnected()

True

Time to grasp a Dataset from OMERO and download a CellProfiler pipeline that is attached to it. Go to the browser, select a dataset and copy the ID.

In [4]:
dataset_id = int(input("Dataset id: "))
dataset = conn.getObject("Dataset", dataset_id)

file_ann_ids = ezomero.get_file_annotation_ids(conn, "Dataset", dataset_id)
for file_ann_id in file_ann_ids:
    if conn.getObject("FileAnnotation", file_ann_id).getFile().getName().endswith(".cppipe"):
        cp_pipeline_path = ezomero.get_file_annotation(conn, file_ann_id, input_dir.name)
        print(f"Downloaded {cp_pipeline_path}")
        break

Downloaded /tmp/tmpmiytnkef/Megane_SpotInNuclei-encours_test_noOUT.cppipe


We create a new pipeline with that file and we remove the first 4 modules. The first 4 modules are in charge of preparing the image data when they are loaded from disk. We don't need them here because we are using OMERO.

In [5]:
pipeline = cp_pipeline.Pipeline()
pipeline.load(cp_pipeline_path)

for i in range(4):
    print('Remove module: ', pipeline.modules()[0].module_name)
    pipeline.remove_module(1)

# TODO: Enable modules
print('Pipeline modules:')
for module in pipeline.modules(False):
    print(module.module_num, module.module_name)

Remove module:  Images
Remove module:  Metadata
Remove module:  NamesAndTypes
Remove module:  Groups
Pipeline modules:
1 CorrectIlluminationCalculate
2 CorrectIlluminationCalculate
3 CorrectIlluminationApply
4 IdentifyPrimaryObjects
5 IdentifyPrimaryObjects
6 RelateObjects
7 MeasureObjectIntensity
8 ConvertObjectsToImage
9 ConvertObjectsToImage
10 SaveImages
11 SaveImages


We can now start feeding images into the pipeline

In [18]:
# Lets create some dataframes to store data on the experiment, images and the different objects measured by cellprofiler
measurement_dfs = {}
for column in pipeline.get_measurement_columns():
    if column[0] not in measurement_dfs.keys():
        measurement_dfs[column[0]] = pd.DataFrame()

# Get the ids of the images we want to analyze
image_ids = ezomero.get_image_ids(conn=conn, dataset=dataset_id, across_groups=False)

# Prepare a progress bar
print("Progress:")
progress_bar = IntProgress(min=0, max=len(image_ids)-1)
display(progress_bar)

# Define which object measurements you want to be stored as a table
objects_to_table = {"nuclei"}

# Define which objects to be saved as masks
objects_to_mask = {"nuclei"}

# Define which objects to be saved as ROI points
objects_to_point = {}
# objects_to_point = {"nuclei"}
# objects_to_point = {"spots"}

# TODO: exclude the possibility to save rois as points and as masks

# Lets collect all images in a dataset and feed them one at a time into the pipeline.
for count, image_id in enumerate(image_ids):
    image, image_pixels = ezomero.get_image(conn, image_id)

    pipeline_copy = pipeline.copy()

    for c in range(image.getSizeC()):
        inject_image_module = InjectImage(f"ch{c}", image_pixels[...,c].squeeze())
        inject_image_module.set_module_num(1)
        pipeline_copy.add_module(inject_image_module)

    measurements = pipeline_copy.run()

    for object_name, _ in measurement_dfs.items():
        if object_name == "Experiment": continue
        if object_name == "Image": continue

        data = {f:measurements.get_measurement(object_name,f) for f in measurements.get_feature_names(object_name)}
        if object_name in objects_to_point:
            data["ROI"] = np.ndarray(shape=(len(data["Number_Object_Number"])), dtype="int")
            for i in range(len(data["Number_Object_Number"])):
                point = ezomero.rois.Point(data["Location_Center_X"][i],
                                           data["Location_Center_Y"][i],
                                           data["Location_Center_Z"][i],
                                           label=f"{object_name}_{data['Number_Object_Number'][i]}")
                point_id = ezomero.post_roi(conn=conn,
                                            image_id=image_id,
                                            shapes=[point],
                                            name=f"{object_name}_{data['Number_Object_Number'][i]}",
                                            description=None)
                data["ROI"][i] = point_id

        elif object_name in objects_to_mask:
            data["ROI"] = np.ndarray(shape=(len(data["Number_Object_Number"])), dtype="int")

            labels = np.load(f"{output_dir.name}/{object_name}.npy")
            print(labels.shape)
            masks = masks_from_labels_image_3d(labels)
            roi_id = create_roi(conn=conn,
                                img=image,
                                shapes=masks,
                                name=f"{object_name}_{data['Number_Object_Number'][i]}")
            data["ROI"][i] = roi_id



        measurement_dfs[object_name] = pd.concat([measurement_dfs[object_name], pd.DataFrame.from_dict(data)], ignore_index=True)

    progress_bar.value = count



Progress:


IntProgress(value=0, max=4)

(2048, 2048)


AxisError: axis 1 is out of bounds for array of dimension 1

In [16]:
for k, v in measurement_dfs.items():
    print(k)
    # v.describe()
    print(v.head())



Experiment
Empty DataFrame
Columns: []
Index: []
Image
Empty DataFrame
Columns: []
Index: []
nuclei
Empty DataFrame
Columns: []
Index: []
spots_unmasked
Empty DataFrame
Columns: []
Index: []
spots
Empty DataFrame
Columns: []
Index: []


In [ ]:
# remove the output directory
output_dir.cleanup()

# and close the connection to the OMERO server
conn.close()